# Phase 1: MNIST Handwritten Digit Dataset

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader

In [2]:
## basic transformations which are mandotory --> i.e converting to tensor and normalization

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


# creating dataset objects - later we will also have dataloader objects

train_dataset = torchvision.datasets.MNIST(
    root = '/data',
    train = True,
    download = True,
    transform = transform
)

test_dataset = torchvision.datasets.MNIST(
    root = '/data',
    train = False,
    download = True,
    transform = transform
)

print (len(train_dataset))
print (len(test_dataset))

60000
10000


In [3]:
# the dataset now contains images which can be accessed using train_dataset[x]
# each image also has a true label whih is the actual class of that label
# currently we are accessing the 69th image --> which turns out to be a zero

sample_image, sample_label = train_dataset[69]

print (sample_image.shape)
print (sample_label)

torch.Size([1, 28, 28])
0


In [4]:
# lets also do it for test dataset
# funny enough we have zero as the 69th image in both the datasets

sample_image, sample_label = test_dataset[69]

print (sample_image.shape)
print (sample_label)

torch.Size([1, 28, 28])
0


# Phase 2: How to Define and Train the Discriminator Model


## Phase 2.1 Discriminator architecture

- Build and Inititalize network - with forward pass capabilities
- also initialize other things like optimizer and loss

In [5]:
class Discriminator(nn.Module):

  def __init__ (self):
    super(Discriminator, self).__init__()

    self.first_conv_block = self._make_first_conv_block()
    self.second_conv_block = self._make_second_conv_block()
    self.final_classification_block = self._make_final_classification_block()


  def _make_first_conv_block(self):

    layers = []

    layers.append(
        nn.Conv2d(
            in_channels = 1,
            out_channels = 64,
            kernel_size = 3,
            stride = 2,
            padding = 1
        )
    )

    layers.append(
        nn.LeakyReLU(
            negative_slope = 0.2,
            inplace = True
        )
    )

    layers.append(
        nn.Dropout(
            p = 0.4
        )
    )

    return nn.Sequential(*layers)





  def _make_second_conv_block(self):

    layers = []

    layers.append(
        nn.Conv2d(
            in_channels = 64,
            out_channels = 64,
            kernel_size = 3,
            stride = 2,
            padding = 1
        )
    )

    layers.append(
        nn.LeakyReLU(
            negative_slope = 0.2,
            inplace = True
        )
    )

    layers.append(
        nn.Dropout(
            p = 0.4
        )
    )

    return nn.Sequential(*layers)






  def _make_final_classification_block (self):

    layers = []

    layers.append(nn.Flatten())

    layers.append(
        nn.Linear(
            in_features = 7*7*64,
            out_features = 1
        )
    )

    layers.append(nn.Sigmoid())

    return nn.Sequential(*layers)



  def forward (self, x):

    x = self.first_conv_block(x)
    x = self.second_conv_block(x)
    x = self.final_classification_block(x)

    return x


In [6]:
def create_discriminator():

  discriminator = Discriminator()


  # in article they have decided to use adam sgd and bce loss


  optimizer = optim.Adam(
      discriminator.parameters(),
      lr = 0.0002,
      betas = (0.5, 0.999)
  )

  criterion = nn.BCELoss()

  return discriminator, optimizer, criterion



# discriminator, optimizer, criterion = create_discriminator()

In [7]:
# total_params = sum(p.numel() for p in discriminator.parameters())
# trainable_params = sum(p.numel() for p in discriminator.parameters() if p.requires_grad)
# print(f"\nTotal Parameters: {total_params:,}")
# print(f"Trainable Parameters: {trainable_params:,}")

## Phase 2.2 Setup Data Sources for Discriminator

### Phase 2.2.1 Function that will provide REAL IMAGES

Approach:
1. Load the data
2. Sample random images from it - along with their labels

In [8]:
dataloader = DataLoader(train_dataset, batch_size = len(train_dataset), shuffle = True)
  # we loaded the complete dataset and NOT A BATCH of images

images, _ = next(iter(dataloader))


In [9]:
def generate_real_samples(n_samples = 100):

  # we already have the dataset loaded in train_dataset from the phase 1 cells
  # therefore we plan to use it

  # dataloader = DataLoader(train_dataset, batch_size = len(train_dataset), shuffle = True)
  # # we loaded the complete dataset and NOT A BATCH of images

  # images, _ = next(iter(dataloader))

  """
  we dont want the labels of the data points because we are not interested in the class of labels as
  1,2,3,4... --> we are just interest in classifying wherther this is real or fake

  Therefore, we will be later setting the labels as "1" for all the images
  which would basically means that:

  "The randomly sampled n_samples images - belong to the class 1 - which is REAL IMAGES"

  """



  # ----->>>>> we have the dataset already - now lets generate randomized n_samples -----


  indices = torch.randint(0, images.shape[0], (n_samples, ))

  train_images = images[indices]

  # we are just having labels as 1 bcoz they are from real image dataset
  train_labels = torch.ones(n_samples, 1)



  return train_images, train_labels


train_real_images, train_real_labels = generate_real_samples()
print(f"Real images shape: {train_real_images.shape}")
print(f"Real images range: [{train_real_images.min():.3f}, {train_real_images.max():.3f}]")
print(f"Real images shape: {train_real_labels.shape}")
print(f"Real images range: [{train_real_labels.min():.3f}, {train_real_labels.max():.3f}]")

Real images shape: torch.Size([100, 1, 28, 28])
Real images range: [-1.000, 1.000]
Real images shape: torch.Size([100, 1])
Real images range: [1.000, 1.000]


### Phase 2.2.2 Function that will provide FAKE IMAGES

**We don’t have a generator model yet, so instead, we can generate images comprised of random pixel values, specifically random pixel values in the range [0,1] like our scaled real images.**

Approach:
1. Random pixels for image

In [10]:
def generate_fake_random_samples(n_samples = 100):


  train_images = torch.rand(n_samples, 1, 28, 28)


  train_labels = torch.zeros(n_samples, 1)

  return train_images, train_labels



train_fake_images, train_fake_labels = generate_fake_random_samples()
print(f"Real images shape: {train_fake_images.shape}")
print(f"Real images range: [{train_fake_images.min():.3f}, {train_fake_images.max():.3f}]")
print(f"Real images shape: {train_fake_labels.shape}")
print(f"Real images range: [{train_fake_labels.min():.3f}, {train_fake_labels.max():.3f}]")

Real images shape: torch.Size([100, 1, 28, 28])
Real images range: [0.000, 1.000]
Real images shape: torch.Size([100, 1])
Real images range: [0.000, 0.000]


## Phase 2.3 Training Discriminator


In [11]:
def train_discriminator(discriminator, optimizer, criterion, n_iter=100, n_batch=256):

    half_batch = n_batch // 2  # Split batch: half real, half fake

    # Track training progress
    real_accuracies = []
    fake_accuracies = []

    discriminator.train()  # Set to training mode (enables dropout)

    for i in range(n_iter):

        # ============================================
        # PHASE 1: TRAIN ON REAL IMAGES
        # ============================================

        # Generate batch of real samples
        X_real, y_real = generate_real_samples(half_batch)

        # Forward pass through discriminator
        pred_real = discriminator(X_real)  # Shape: [half_batch, 1]

        # Calculate loss: how wrong are predictions on real images?
        # MATHEMATICAL INSIGHT: BCE loss = -[y*log(p) + (1-y)*log(1-p)]
        # For real images (y=1): loss = -log(p), minimized when p→1
        loss_real = criterion(pred_real, y_real)

        # Calculate accuracy before updating
        # TECHNICAL NOTE: Convert probabilities to binary predictions
        pred_real_binary = (pred_real > 0.5).float()
        real_acc = (pred_real_binary == y_real).float().mean().item()

        # Backpropagation and optimization
        optimizer.zero_grad()  # Clear previous gradients
        loss_real.backward()   # Compute gradients
        optimizer.step()       # Update weights



        # ============================================
        # PHASE 2: TRAIN ON FAKE IMAGES
        # ============================================



        # Generate batch of fake samples
        X_fake, y_fake = generate_fake_random_samples(half_batch)

        # Forward pass
        pred_fake = discriminator(X_fake)

        # Calculate loss on fake images
        # For fake images (y=0): loss = -log(1-p), minimized when p→0
        loss_fake = criterion(pred_fake, y_fake)

        # Calculate accuracy
        pred_fake_binary = (pred_fake > 0.5).float()
        fake_acc = (pred_fake_binary == y_fake).float().mean().item()

        # Backpropagation
        optimizer.zero_grad()
        loss_fake.backward()
        optimizer.step()

        # ============================================
        # MONITORING AND LOGGING
        # ============================================

        real_accuracies.append(real_acc)
        fake_accuracies.append(fake_acc)

        # Print progress every 10 iterations
        if (i + 1) % 10 == 0:
            print(f'Iteration {i+1:>3d}: Real={real_acc*100:5.1f}% Fake={fake_acc*100:5.1f}%')

    return real_accuracies, fake_accuracies


In [12]:
# Execute the training
print("Training Discriminator...")

discriminator, optimizer, criterion = create_discriminator()

real_accs, fake_accs = train_discriminator(
    discriminator=discriminator,
    optimizer=optimizer,
    criterion=criterion,
    n_iter=100,
    n_batch=256
)

print(f"\nFinal Performance:")
print(f"Real Image Accuracy: {real_accs[-1]*100:.1f}%")
print(f"Fake Image Accuracy: {fake_accs[-1]*100:.1f}%")


Training Discriminator...
Iteration  10: Real=100.0% Fake=100.0%
Iteration  20: Real=100.0% Fake=100.0%
Iteration  30: Real=100.0% Fake=100.0%
Iteration  40: Real=100.0% Fake=100.0%
Iteration  50: Real=100.0% Fake=100.0%
Iteration  60: Real=100.0% Fake=100.0%
Iteration  70: Real=100.0% Fake=100.0%
Iteration  80: Real=100.0% Fake=100.0%
Iteration  90: Real=100.0% Fake=100.0%
Iteration 100: Real=100.0% Fake=100.0%

Final Performance:
Real Image Accuracy: 100.0%
Fake Image Accuracy: 100.0%


# PHASE 3: How to Define and Use the Generator Model